In [2]:
%pip install xgboost==2.0.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 47.9 MB/s  0:00:05m0:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from pathlib import Path

# 프로젝트 경로
PROJECT_ROOT = Path.cwd()
CODE_DIR = PROJECT_ROOT / "code"
DATA_DIR = PROJECT_ROOT / "data"

print(f"Project root: {PROJECT_ROOT}")
print(f"Code directory: {CODE_DIR}")
print(f"Data directory: {DATA_DIR}")

Project root: /home/ec2-user/SageMaker/kunops-staff
Code directory: /home/ec2-user/SageMaker/kunops-staff/code
Data directory: /home/ec2-user/SageMaker/kunops-staff/data


In [5]:
# =============================================================================
# AWS 설정
# =============================================================================
import sagemaker
import boto3
from sagemaker.estimator import Estimator

AWS_REGION = "us-east-1"

# SageMaker Session 생성
boto_session = boto3.Session(region_name=AWS_REGION)
sm_session = sagemaker.Session(boto_session=boto_session)

SAGEMAKER_ROLE = sagemaker.get_execution_role()

# S3 경로
S3_BUCKET = "kunops-staff-exercise"
S3_DATA_PREFIX = "data"
S3_MODEL_PREFIX = "models"
S3_DATA_PATH = f"s3://{S3_BUCKET}/{S3_DATA_PREFIX}"

# ECR 이미지
ECR_ACCOUNT_ID = "155954279556"
IMAGE_NAME = "kunops-sm/xgboost-training"
IMAGE_TAG = "0.9.2-cpu-py312"
IMAGE_URI = f"{ECR_ACCOUNT_ID}.dkr.ecr.{AWS_REGION}.amazonaws.com/{IMAGE_NAME}:{IMAGE_TAG}"

print(f"AWS Region: {AWS_REGION}")
print(f"SageMaker Role: {SAGEMAKER_ROLE}")
print(f"S3 Bucket: {S3_BUCKET}")
print(f"S3 Data Path: {S3_DATA_PATH}")
print(f"ECR Image URI: {IMAGE_URI}")

AWS Region: us-east-1
SageMaker Role: arn:aws:iam::155954279556:role/service-role/AmazonSageMaker-ExecutionRole-20260123T111313
S3 Bucket: kunops-staff-exercise
S3 Data Path: s3://kunops-staff-exercise/data
ECR Image URI: 155954279556.dkr.ecr.us-east-1.amazonaws.com/kunops-sm/xgboost-training:0.9.2-cpu-py312


In [6]:
# =============================================================================
# XGBoost 하이퍼파라미터 설정
# =============================================================================

hyperparameters = {
    # XGBoost 파라미터
    "n-estimators": 200,           # 부스팅 라운드 수
    "max-depth": 4,                # 트리 최대 깊이
    "learning-rate": 0.05,          # 학습률
    "early-stopping-rounds": 20,   # Early stopping
}

print("Hyperparameters:")
for key, value in hyperparameters.items():
    print(f"   {key}: {value}")

Hyperparameters:
   n-estimators: 200
   max-depth: 4
   learning-rate: 0.05
   early-stopping-rounds: 20


In [7]:
# 모델 아티팩트 저장 경로
# Instance    || 시간당 가격     || vCPU  || 메모리
# ml.t3.large || ~$0.10/hour  || 2     || 8 GiB
# ml.m5.large || $0.115/hour || 2     || 8 GiB

INSTANCE_TYPE = "ml.t3.large"
MODEL_OUTPUT_PATH = f"s3://{S3_BUCKET}/{S3_MODEL_PREFIX}"


tags: list[dict[str, str]] = [
    {"Key": "Project", "Value": "edu"},
    {"Key": "Environment", "Value": "test"},
    {"Key": "Owner", "Value": "kunops"},
]

# Estimator 생성
estimator = Estimator(
    image_uri=IMAGE_URI,
    entry_point="train_xgboost.py",
    source_dir=str(CODE_DIR),                # 코드 디렉토리
    role=SAGEMAKER_ROLE,                     # IAM Role
    instance_type=INSTANCE_TYPE,             # 인스턴스 타입
    instance_count=1,
    hyperparameters=hyperparameters,
    sagemaker_session=sm_session,
    tags=tags,  # type: ignore[arg-type]
    output_path=MODEL_OUTPUT_PATH,           # 모델 저장 경로
    
    # 환경변수
    # environment={
    #     "SAGEMAKER_PROGRAM": "train_xgboost.py", # Defalut로 train 지정되어 있음
    # },
    
    # Job 이름 prefix
    base_job_name="kunops-titanic-xgboost",
    
    # 최대 실행 시간 (초) - 30분
    max_run=1800,
)

print("Estimator created")
print(f"   Image URI: {IMAGE_URI}")
print(f"   Entry point: {estimator.entry_point}")
print(f"   Instance type: {INSTANCE_TYPE}")
print(f"   Model output: {MODEL_OUTPUT_PATH}")

Estimator created
   Image URI: 155954279556.dkr.ecr.us-east-1.amazonaws.com/kunops-sm/xgboost-training:0.9.2-cpu-py312
   Entry point: train_xgboost.py
   Instance type: ml.t3.large
   Model output: s3://kunops-staff-exercise/models


In [8]:
# S3 데이터 경로 설정
train_data = f"{S3_DATA_PATH}/train.csv"
validation_data = f"{S3_DATA_PATH}/validation.csv"

print("Starting SageMaker Training Job")
print("=" * 60)
print(f"Train data: {train_data}")
print(f"Validation data: {validation_data}")
print("=" * 60)

# 학습 실행
estimator.fit(
    inputs={
        "train": train_data,
        "validation": validation_data,
    },
    wait=True,
    logs="All",
)

print("=" * 60)
print("Training completed!")

INFO:sagemaker:Creating training-job with name: kunops-titanic-xgboost-2026-02-03-05-54-35-559


Starting SageMaker Training Job
Train data: s3://kunops-staff-exercise/data/train.csv
Validation data: s3://kunops-staff-exercise/data/validation.csv
2026-02-03 05:54:39 Starting - Starting the training job...
2026-02-03 05:54:53 Starting - Preparing the instances for training...
2026-02-03 05:55:22 Downloading - Downloading input data...
2026-02-03 05:56:07 Training - Training image download completed. Training in progress..2026-02-03 05:56:09,792 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2026-02-03 05:56:09,793 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2026-02-03 05:56:09,806 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2026-02-03 05:56:09,807 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2026-02-03 05:56:09,819 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2026-02-03 05:56:09,820 

In [9]:
# 모델 다운로드 (선택)
DOWNLOAD_MODEL = True

if DOWNLOAD_MODEL:
    import tarfile
    import json
    from sagemaker.s3 import S3Downloader
    
    # 모델 다운로드 경로
    local_model_dir = PROJECT_ROOT / "models" / "sagemaker"
    local_model_dir.mkdir(parents=True, exist_ok=True)
    
    # S3에서 다운로드
    model_uri = estimator.model_data
    print(model_uri)
    S3Downloader.download(
        s3_uri=model_uri,
        local_path=str(local_model_dir),
        sagemaker_session=sm_session
    )
    
    # tar.gz 압축 해제
    tar_path = local_model_dir / "model.tar.gz"
    if tar_path.exists():
        with tarfile.open(tar_path, "r:gz") as tar:
            tar.extractall(local_model_dir, filter="data")
        print(f"✅ Model downloaded and extracted to: {local_model_dir}")
        
        # 다운로드된 파일 확인
        print("\nDownloaded files:")
        for f in local_model_dir.iterdir():
            print(f"   - {f.name}")
        
        # metrics.json 출력
        metrics_path = local_model_dir / "metrics.json"
        if metrics_path.exists():
            with open(metrics_path) as f:
                metrics = json.load(f)
            
            print("\n" + "=" * 50)
            print("📊 Model Performance Metrics")
            print("=" * 50)
            for key, value in metrics.items():
                bar = "█" * int(value * 40)
                print(f"   {key:12s}: {value:.4f} |{bar}")
            print("=" * 50)
        else:
            print("\n⚠️ metrics.json not found")
else:
    print("Model download skipped. Set DOWNLOAD_MODEL=True to download.")

s3://kunops-staff-exercise/models/kunops-titanic-xgboost-2026-02-03-05-54-35-559/output/model.tar.gz
✅ Model downloaded and extracted to: /home/ec2-user/SageMaker/kunops-staff/models/sagemaker

Downloaded files:
   - model.tar.gz
   - metrics.json
   - xgboost_model.pkl
   - label_encoders.pkl

📊 Model Performance Metrics
   accuracy    : 0.8315 |█████████████████████████████████
   precision   : 0.8654 |██████████████████████████████████
   recall      : 0.6618 |██████████████████████████
   f1          : 0.7500 |██████████████████████████████
   roc_auc     : 0.8931 |███████████████████████████████████


In [10]:
# 다운로드한 모델로 test.csv 예측
import joblib
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 모델 경로
local_model_dir = PROJECT_ROOT / "models" / "sagemaker"
model_path = local_model_dir / "xgboost_model.pkl"
encoders_path = local_model_dir / "label_encoders.pkl"

if not model_path.exists():
    print("❌ 모델 파일이 없습니다. 먼저 위 셀에서 DOWNLOAD_MODEL=True로 다운로드하세요.")
else:
    # 1. 모델 & 인코더 로드
    print("=" * 60)
    print("🔄 모델 로딩...")
    model = joblib.load(model_path)
    label_encoders = joblib.load(encoders_path)
    print(f"   ✅ Model loaded: {model_path.name}")
    print(f"   ✅ Encoders loaded: {encoders_path.name}")

    # 2. Test 데이터 로드
    print("\n🔄 Test 데이터 로딩...")
    test_df = pd.read_csv(DATA_DIR / "test.csv")
    print(f"   ✅ Test data: {len(test_df)} rows")

    # 3. 전처리 (학습 시와 동일하게)
    print("\n🔄 전처리 중...")
    
    # 타겟 분리
    y_test = test_df["survived"].to_numpy()
    
    # 피처 선택
    feature_cols = ["pclass", "sex", "age", "sibsp", "parch", "fare", "embarked", "alone"]
    X_test = test_df[feature_cols].copy()
    
    # 결측치 처리
    X_test["age"] = X_test["age"].fillna(X_test["age"].median())
    X_test["fare"] = X_test["fare"].fillna(X_test["fare"].median())
    X_test["embarked"] = X_test["embarked"].fillna("S")
    
    # 범주형 인코딩 (저장된 인코더 사용)
    for col in ["sex", "embarked"]:
        if col in label_encoders:
            X_test[col] = label_encoders[col].transform(X_test[col].astype(str))
    
    # Boolean to int
    X_test["alone"] = X_test["alone"].astype(int)
    
    print(f"   ✅ Features: {list(X_test.columns)}")

    # 4. 예측
    print("\n🔄 예측 수행 중...")
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # 5. 성능 평가
    print("\n" + "=" * 60)
    print("📊 Test 데이터 예측 결과")
    print("=" * 60)
    
    metrics = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "Precision": precision_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "ROC-AUC": roc_auc_score(y_test, y_pred_proba),
    }
    
    for name, value in metrics.items():
        bar = "█" * int(value * 40)
        print(f"   {name:12s}: {value:.4f} |{bar}")
    
    # 6. Classification Report
    print("\n" + "=" * 60)
    print("📋 Classification Report")
    print("=" * 60)
    print(classification_report(y_test, y_pred, target_names=["Died (0)", "Survived (1)"]))
    
    # 7. Confusion Matrix
    print("📋 Confusion Matrix")
    print("=" * 60)
    cm = confusion_matrix(y_test, y_pred)
    print(f"                 Predicted")
    print(f"                 Died  Survived")
    print(f"   Actual Died    {cm[0][0]:4d}    {cm[0][1]:4d}")
    print(f"   Actual Surv    {cm[1][0]:4d}    {cm[1][1]:4d}")
    print("=" * 60)

🔄 모델 로딩...
   ✅ Model loaded: xgboost_model.pkl
   ✅ Encoders loaded: label_encoders.pkl

🔄 Test 데이터 로딩...
   ✅ Test data: 90 rows

🔄 전처리 중...
   ✅ Features: ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'alone']

🔄 예측 수행 중...

📊 Test 데이터 예측 결과
   Accuracy    : 0.7778 |███████████████████████████████
   Precision   : 0.8261 |█████████████████████████████████
   Recall      : 0.5429 |█████████████████████
   F1 Score    : 0.6552 |██████████████████████████
   ROC-AUC     : 0.8548 |██████████████████████████████████

📋 Classification Report
              precision    recall  f1-score   support

    Died (0)       0.76      0.93      0.84        55
Survived (1)       0.83      0.54      0.66        35

    accuracy                           0.78        90
   macro avg       0.79      0.74      0.75        90
weighted avg       0.79      0.78      0.77        90

📋 Confusion Matrix
                 Predicted
                 Died  Survived
   Actual Died      51       4
  

/home/ec2-user/anaconda3/envs/python3/lib/python3.12/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.4.2 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
